In [ ]:
from colabcode import ColabCode

In [ ]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

np.set_printoptions(suppress=True)

In [ ]:
# Object points in 3D
GRID_SHAPE = (8,5)
objp = np.zeros((GRID_SHAPE[0]*GRID_SHAPE[1],3), np.float32)
objp[:,:2] = np.mgrid[0:GRID_SHAPE[0], 0:GRID_SHAPE[1]].T.reshape(-1,2)
objp *= 30 # One square on my grid has 20mm

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
flags = cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_NORMALIZE_IMAGE + cv2.CALIB_CB_FAST_CHECK


In [ ]:
FOLDER_LEFT = "stereo_left/"
FOLDER_RIGHT = "stereo_right/"


In [ ]:
fnames = os.listdir(FOLDER_LEFT)
obj_pts = []
img_pts_left = []
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in fnames:
    print(f"processing {fname}")
    img_BGR = cv2.imread (FOLDER_LEFT+fname)
    img_RGB = cv2.cvtColor( img_BGR, cv2.COLOR_BGR2RGB)
    img_GRAY = cv2.cvtColor( img_RGB, cv2.COLOR_BGR2GRAY)
    arr_l = np.array (img_GRAY)
    # arr = np.array(img_GRAY)

    ret, corners = cv2.findChessboardCorners(img_GRAY, GRID_SHAPE, flags)

    arr_vis = cv2.drawChessboardCorners(img_GRAY, GRID_SHAPE, corners, ret)
    plt.imshow(arr_vis, cmap='gray')
    plt.show()

    if ret:
        corners_subpix = cv2.cornerSubPix(img_GRAY,corners,(11,11),(-1,-1),criteria)
        obj_pts.append(objp)
        img_pts_left.append(corners_subpix)



In [ ]:
ret, K_l, dist_coeff_l, rvecs, tvecs = cv2.calibrateCamera(obj_pts, img_pts_left, (img_GRAY.shape[1], img_GRAY.shape[0]), None,None)

In [ ]:
ret

In [ ]:
K_l

In [ ]:
dist_coeff_l

In [ ]:
img = Image.open(FOLDER_LEFT+"5.png")
arr = np.array(img)

arr_corr = cv2.undistort(arr, K_l, dist_coeff_l, None, K_l)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(arr, cmap='gray')
ax2.imshow(arr_corr, cmap='gray')
plt.show()


In [ ]:
fnames = os.listdir(FOLDER_RIGHT)
obj_pts = []
img_pts_right  = []
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

for fname in fnames:
    print(f"processing {fname}")
    img_BGR = cv2.imread (FOLDER_RIGHT+fname)
    img_RGB = cv2.cvtColor( img_BGR, cv2.COLOR_BGR2RGB)
    img_GRAY = cv2.cvtColor( img_RGB, cv2.COLOR_BGR2GRAY)
    arr_r = np.array (img_GRAY)
    # arr = np.array(img_GRAY)

    ret, corners = cv2.findChessboardCorners(img_GRAY, GRID_SHAPE, flags)

    arr_vis = cv2.drawChessboardCorners(img_GRAY, GRID_SHAPE, corners, ret)
    plt.imshow(arr_vis, cmap='gray')
    plt.show()

    if ret:
        corners_subpix = cv2.cornerSubPix(img_GRAY,corners,(11,11),(-1,-1),criteria)
        obj_pts.append(objp)
        img_pts_right.append(corners_subpix)

In [ ]:
ret, K_r, dist_coeff_r, rvecs, tvecs = cv2.calibrateCamera(obj_pts , img_pts_right , (img_GRAY.shape[1], img_GRAY.shape[0]), None, None)

In [ ]:
K_r

In [ ]:
K_l

In [ ]:
dist_coeff_r

In [ ]:
dist_coeff_l

In [ ]:
img = Image.open(FOLDER_RIGHT+"5.png")
arr = np.array(img)

arr_corr = cv2.undistort(arr, K_r, dist_coeff_r, None, K_l)

fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,10))
ax1.imshow(arr, cmap='gray')
ax2.imshow(arr_corr, cmap='gray')
plt.show()

In [ ]:
np.save("K_l.npy", K_l)
np.save("K_r.npy", K_r)

np.save("dist_coeff_l.npy", dist_coeff_l)
np.save("dist_coeff_r.npy", dist_coeff_r)

In [ ]:
flags = 0
flags |= cv2.CALIB_FIX_INTRINSIC

criteria_stereo= (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

# This step is performed to transformation between the two cameras and calculate Essential and Fundamenatl matrix
retS, K_l, dist_coeff_l, K_r, dist_coeff_r, Rot, Trns, Emat, Fmat = cv2.stereoCalibrate(
    obj_pts,
    img_pts_left,
    img_pts_right,
    K_l, dist_coeff_l,
    K_r, dist_coeff_r,
    img_GRAY.shape[::-1],
    criteria_stereo,
    flags)


In [ ]:
rect_l, rect_r, proj_mat_l, proj_mat_r, Q, roiL, roiR = cv2.stereoRectify(K_l, dist_coeff_l, K_r, dist_coeff_r, img_GRAY.shape[::-1], Rot, Trns, 1 ,(0,0))

In [ ]:
left_stereo_maps = cv2.initUndistortRectifyMap(K_l, dist_coeff_l, rect_l, proj_mat_l,
                                             img_GRAY.shape[::-1], cv2.CV_16SC2)
right_stereo_maps = cv2.initUndistortRectifyMap(K_r, dist_coeff_r, rect_r, proj_mat_r,
                                              img_GRAY.shape[::-1], cv2.CV_16SC2)


In [ ]:
arr_l_rect = cv2.remap(arr_l, left_stereo_maps[0],left_stereo_maps[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
arr_r_rect = cv2.remap(arr_r, right_stereo_maps[0],right_stereo_maps[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)

In [ ]:
for fname_l, fname_r in zip(sorted(os.listdir(FOLDER_LEFT)), sorted(os.listdir(FOLDER_RIGHT))):

    print(f"Images L: {fname_l}, R: {fname_r}")

    img_l, img_r = Image.open(FOLDER_LEFT + fname_l), Image.open(FOLDER_RIGHT + fname_r)
    arr_l, arr_r = np.array(img_l), np.array(img_r)

    arr_l_rect = cv2.remap(arr_l, left_stereo_maps[0],left_stereo_maps[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
    arr_r_rect = cv2.remap(arr_r, right_stereo_maps[0],right_stereo_maps[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)

    fig, [(ax1, ax2), (ax3, ax4)] = plt.subplots(2, 2, figsize=(10,3))

    # before
    ax1.imshow(arr_l[100:200,:], cmap='binary')
    ax2.imshow(arr_r[100:200,:], cmap='binary')

    ax1.title.set_text('Original L')
    ax2.title.set_text('Original R')


    # after
    ax3.imshow(arr_l_rect[100:200,:], cmap='gray')
    ax4.imshow(arr_r_rect[100:200,:], cmap='gray')

    ax3.title.set_text('Rectified L')
    ax4.title.set_text('Rectified R')
    plt.show()

In [ ]:
cv_file = cv2.FileStorage("rectify_map_imx219_160deg_1080p_new.yaml", cv2.FILE_STORAGE_WRITE)
cv_file.write("map_l_1", left_stereo_maps[0])
cv_file.write("map_l_2", left_stereo_maps[1])
cv_file.write("map_r_1", right_stereo_maps[0])
cv_file.write("map_r_2", right_stereo_maps[1])
cv_file.release()

In [ ]:
map_ref_x, map_ref_y = np.mgrid[0:1080:1, 0:1920:1]

In [ ]:
min_disp = 0
num_disp = 100
window_size = 15

# объект StereoSGBM для вычисления disparity map
stereo = cv2.StereoSGBM_create(numDisparities=16, blockSize=15)

fig, axs = plt.subplots(2, 2, figsize=(20,10))


# print(f"Images L: {fname_l}, R: {fname_r}"
img_l, img_r = Image.open(FOLDER_LEFT + 'left_cam_img.png'), Image.open(FOLDER_RIGHT + "right_cam_img.png")
arr_l, arr_r = np.array(img_l), np.array(img_r)

arr_l_rect = cv2.remap(arr_l, left_stereo_maps[0],left_stereo_maps[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)
arr_r_rect = cv2.remap(arr_r, right_stereo_maps[0],right_stereo_maps[1], cv2.INTER_LANCZOS4, cv2.BORDER_CONSTANT, 0)

# Вычислите disparity map
disparity_map = stereo.compute( arr_l_rect, arr_r_rect)

# before
axs[0][0].imshow(arr_l_rect)
axs[0][1].imshow(arr_r_rect)

axs[0][0].title.set_text('Rectified L')
axs[0][0].title.set_text('Rectified R')

# after
axs[1][0].imshow((disparity_map / 16).astype(np.uint8))
axs[1][0].title.set_text('Disparity Map')
plt.show()




# depth_map_SGBM = (6.5 * focal_length) / (disparity_map + 1e-6)



# cv2.imshow("Disparity Map", (disparity_map / 16).astype(np.uint8))  # Для удобства, масштабируем на 16
# cv2.imshow("Depth Map", depth_map.astype(np.float32))
# # Показать карту глубины
# cv2.imshow('Disparity', disparity / disparity.max())